# 第八章之Python函数定义的深度解析
___

<font color="red">注意：以下部分内容只能在Python3.8以上版本兼容。</font>

我们在查看内置的函数的时候，常常会看到这么一些定义方式：
- `def func(a, b, /)`
- `def func(a=None, b=None, *)`
- `def func(*args, **kwargs)`
- `def func(a, b, /, c, d, *, e=None, f=None)`

那么，这些函数定义的方式有什么讲究的么？

## 1. 纯位置参数（Positional-only arguments）

源自PEP-570。目前看来，只能支持3.8以上的Python版本。

In [1]:
def mysum(a, b, /):
    return a + b

这种类型的位置参数在调用的时候只能传入值，不能用`name=value`的方式传入参数：

In [2]:
mysum(2,4)

6

In [3]:
mysum(2,b=4)

TypeError: mysum() got some positional-only arguments passed as keyword arguments: 'b'

## 2. 纯关键词参数

同样的，`*`则表示之后的参数为关键词参数，不能是位置参数，之前的可为未知参数或关键词参数：

In [4]:
def mysum(*, a, b):
    return a + b

In [5]:
mysum(3,5)

TypeError: mysum() takes 0 positional arguments but 2 were given

In [6]:
mysum(a=3, b=4)

7

In [8]:
mysum(b=4, a=5)

9

In [7]:
mysum(3, b=3)

TypeError: mysum() takes 0 positional arguments but 1 positional argument (and 1 keyword-only argument) were given

## 3. 两种参数的结合

我们还可以将位置参数和关键词参数结合一下：

In [9]:
def mysum(a, b, /, c, d, *, e, f):
    return a + b + c + d + e + f

In [10]:
mysum(3,4,5,6,7,8)

TypeError: mysum() takes 4 positional arguments but 6 were given

In [11]:
mysum(3,4,5,6,e=7,f=8)

33

In [12]:
mysum(3,4,5,d=6,e=7,f=8)

33

In [13]:
mysum(3,4,c=5,d=6,e=7,f=8)

33

In [14]:
mysum(3,4,d=5,c=6,e=7,f=8)

33

In [15]:
mysum(3,4,d=5,6,e=7,f=8)

SyntaxError: positional argument follows keyword argument (<ipython-input-15-1362c59b9fa8>, line 1)

## 4. 不定长位置参数

不定长的位置参数其实是非常常见的，例如在bash中我们用`$@`或`$*`表示所有位置参数，而在C语言中我们也有办法可以保证函数的参数个数不固定的情况，而在Python中则用`*args`表示不定长位置参数`args`，被解析为元组。

In [22]:
def mysum(*args):
    s = 0
    print(args)
    for i in args:
        s += i
    return s

In [23]:
mysum(1,2,3,4,5,6)

(1, 2, 3, 4, 5, 6)


21

也可以将一个元组或者list解包后计算：

In [26]:
t = (1,2,3,4,5)

In [27]:
mysum(*t)

(1, 2, 3, 4, 5)


15

In [28]:
li = [i for i in range(20)]

In [29]:
mysum(*li)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)


190

## 5. 不定长关键词参数

不定长关键词参数则可用`**kwargs`表示：

In [30]:
def mysum(**kwargs):
    s = 0
    print(kwargs)
    for i in kwargs.values():
        s += i
    return s

然后我们就可以用`name=value`的方式将多个值给传递进去，参数被作为一个字典：

In [31]:
mysum(a=2, b=2, c=5)

{'a': 2, 'b': 2, 'c': 5}


9

In [32]:
mysum(a=5, b=3, c=3, x=6, y=1)

{'a': 5, 'b': 3, 'c': 3, 'x': 6, 'y': 1}


18

也可以将一个字典解包后传入函数：

In [33]:
args = {"a":1, "b":3, "x":4, "z":8}

In [37]:
mysum(**args)

{'a': 1, 'b': 3, 'x': 4, 'z': 8}


16

## 6. 带默认值的参数

带默认值的参数只能放在定长位置参数和不定长位置参数之后，不定长字典参数之前

In [38]:
def mysum(a=1, b=2, *c, d=5, e=6, **f):
    s = a + b + d + e
    for i in c:
        s += i
    for j in f.values():
        s += j
    return s

这里的默认值参数就是`d=5`和`e=6`，如果不给这两个参数重新赋值，就会采用这两个默认值进行计算。

In [39]:
mysum(1,2,3,4,5,x=3,y=5,z=7)

41

这里参数`c`解析为`(3,4,5)`，`d`和`e`采用了默认的参数值5和6。

In [40]:
mysum(1,2,3,4,5,d=9,x=3,y=5,z=7)

45

In [ ]:
这里参数`c`解析为`(3,4,5)`，`d`采用了传入的值9，而`e`采用了默认的值6。

## 小结

一个具有完整的参数列表函数为：

In [42]:
def func(a,b,*args,c=4,d=5,**kwargs):
    pass